In [37]:
# 예제 1
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

In [38]:
park = Customer('Park', 100)
park

Customer(name='Park', fidelity=100)

In [39]:
class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity

In [40]:
class Order:
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

class Promotion(ABC):
    @abstractmethod
    def discount(self, order):
        pass

class FidelityPromo(Promotion):
    def discount(self, order):
        return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

class BulkItemPromo(Promotion):
    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * .1
        return discount

class LargeOrderPromo(Promotion):
    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * 0.07
        return 0

In [41]:
# 예제 2
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [
    LineItem('banana', 4, .5),
    LineItem('apple', 10, 1.5),
    LineItem('watermellon', 5, 5.0)
]

In [42]:
Order(joe, cart, FidelityPromo())

<Order total: 42.00 due: 42.00>

In [10]:
Order(ann, cart, FidelityPromo())

<Order total: 42.00 due: 39.90>

In [43]:
banana_cart = [
    LineItem('banana', 30, .5),
    LineItem('apple', 10, 1.5)
]
Order(joe, banana_cart, BulkItemPromo())

<Order total: 30.00 due: 28.50>

In [44]:
Long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]
Order(joe, Long_order, LargeOrderPromo())

<Order total: 10.00 due: 9.30>

In [13]:
Order(joe, cart, LargeOrderPromo())

<Order total: 42.00 due: 42.00>

In [45]:
# 예제 3
class Order:
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

In [62]:
def fidelity_promo(order):
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

def bulk_item_promo(order):
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

def large_order_promo(order):
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * 0.07
    return 0

In [63]:
Order(joe, cart, fidelity_promo)

<Order total: 42.00 due: 42.00>

In [64]:
Order(ann, cart, fidelity_promo)

<Order total: 42.00 due: 39.90>

In [65]:
Order(joe, banana_cart, bulk_item_promo)

<Order total: 30.00 due: 28.50>

In [66]:
Order(ann, Long_order, large_order_promo)

<Order total: 10.00 due: 9.30>

In [67]:
promos = [fidelity_promo, bulk_item_promo, large_order_promo]

def best_promo(order):
    return max(promo(order) for promo in promos)

In [68]:
Order(joe, long_order, best_promo)

<Order total: 10.00 due: 9.30>

In [69]:
Order(joe, banana_cart, best_promo)

<Order total: 30.00 due: 28.50>

In [70]:
Order(ann, cart, best_promo)

<Order total: 42.00 due: 39.90>

In [71]:
globals().keys()

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', '_', '__', '___', '_i', '_ii', '_iii', '_i1', 'ABC', 'abstractmethod', '_i2', 'namedtuple', 'Customer', '_i3', 'park', '_3', '_i4', 'LineItem', '_i5', 'Order', '_i6', 'greet', 'greeting', 'fun', '_i7', 'Promotion', 'FidelityPromo', 'BulkItemPromo', 'LargeOrderPromo', '_i8', 'joe', 'ann', 'cart', '_i9', '_9', '_i10', '_10', '_i11', 'banana_cart', '_11', '_i12', 'Long_order', '_12', '_i13', '_13', '_i14', '_i15', '_i16', '_16', '_i17', '_i18', '_18', '_i19', '_19', '_i20', '_20', '_i21', '_i22', '_i23', 'long_order', '_23', '_i24', '_24', '_i25', '_i26', '_26', '_i27', '_i28', '_28', '_i29', '_i30', '_i31', '_31', '_i32', '_i33', '_i34', '_i35', '_i36', '_i37', '_i38', '_38', '_i39', '_i40', '_i41', '_i42', '_42', '_i43', '_43', '_i44', '_44', '_i45', '_i46', '_i47', '_i48', '_i49', '_i50', 'promos', 'best_promo', '_i51

In [72]:
globals()['best_promo']

<function __main__.best_promo(order)>

In [73]:
promos = [globals()[name] for name in globals() if name.endswith('_promo') and name != 'best_promo']
promos

[<function __main__.fidelity_promo(order)>,
 <function __main__.bulk_item_promo(order)>,
 <function __main__.large_order_promo(order)>]